# Welcome to the MNIST KAN Performance check
So, the plan is, get the MNIST dataset, and try to make a KAN work on it, and more importantly, seeing how good of a performance can be achieved.

Some boiler plate stuff may be borrowed from the assignments, etc.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch
from torch import nn
from kan import *

from torchvision.datasets import MNIST

In [2]:
# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset Setup
mnist_trainset = MNIST("./temp/", train=True, download=True)
mnist_testset = MNIST("./temp/", train=False, download=True)

In [3]:
# Finalise dataset setup
train_x = mnist_trainset.data.view(-1, 28*28)
train_y = torch.tensor((pd.get_dummies(pd.Series(mnist_trainset.targets.numpy())).map(lambda x: 1 if x == True else 0)).values)

test_x = mnist_testset.data.view(-1, 28*28)
test_y = torch.tensor((pd.get_dummies(pd.Series(mnist_testset.targets.numpy())).map(lambda x: 1 if x == True else 0)).values)

# Setup the training data for the KAN
# https://kindxiaoming.github.io/pykan/Example/Example_4_classfication.html

dataset = {}
dataset['train_input'] = train_x.type(torch.float32).to(device)
dataset['test_input'] = test_x.type(torch.float32).to(device)
dataset['train_label'] = train_y.type(torch.float32).to(device)
dataset['test_label'] = test_y.type(torch.float32).to(device)

def test_acc():
    #v1 = torch.argmax(model(dataset['test_input']), dim=1)
    v1 = []
    lim = 10
    dsize = len(dataset['test_input'])//lim
    for i in range(lim):
        v1.extend(torch.argmax(model(dataset['test_input'][i*dsize:(i+1)*dsize]), dim=1))
    res = 0
    for i in range(len(dataset['test_label'])):
        res += (dataset['test_label'][i, v1[i]] == 1)
    return (res / len(dataset['test_label']))

In [4]:
# Setup the pykan model:
model = KAN(width=[28*28, 240, 60, 10], grid=2, k=2, seed=42, device=device)

checkpoint directory created: ./model
saving model version 0.0


In [5]:
#Train & Test the KAN Model on MNIST
# Inspired by this paper: https://arxiv.org/html/2411.05296v1
# https://arxiv.org/pdf/2404.19756 -- Page 7 for the bound
results = model.fit(dataset, opt="Adam", steps=10000, batch=1000, lr=0.0001, loss_fn=torch.nn.CrossEntropyLoss());
print(f'Test Accuracy: {test_acc()}')

| train_loss: 4.41e-03 | test_loss: 2.74e-01 | reg: 3.09e+03 | : 100%|█| 10000/10000 [22:18<00:00,  


saving model version 0.1
Test Accuracy: 0.974299967288971
